<a href="https://colab.research.google.com/github/pirri87/GOB_Medio_Amb/blob/main/APCA_GENERAL_ALL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries & Load Drive

In [ ]:
# libraries
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Funciones

### Código
Cada sheet estara en un dataframe para ello tengo que hacer lo siguiente:
*   Saber su apca_id
*   Cogo los datos que necesito (Filas y columnas)
*   Traspongo las columnas en filas
*   Añado el apca_id.



In [ ]:
def take_apca_id(df_apca_id):
  df_apca_id.head(0) # En el encabezado hay información que es dato y no keys, para ello hago estos pasos
  mydict = df_apca_id.head(0).to_dict() # Lo transformo para poder manipularlo
  apca_long = list(mydict.keys())[1] # Obtengo el dato que necesito que es el id del apca
  #print(apca_long)
  return apca_long[0:8]  # Corto el final para que no salga letras ni escpacios

In [ ]:
def manipulador(xls,end, alcance):
  list_df_all = []
  for i in range(end):
  #for i in range(2):
    df = pd.read_excel(xls,i)# Load sheet, star in 0 , con la i recorro cada uno de los sheet
    apca_id = take_apca_id(df)
    #print(apca_id)
    df = df.iloc[0:9 , 0:2] # Me quedo con las columnas y filas que yo quiero.
    df = df.transpose() # Transpongo las filas a columna
    df = df.rename(columns= df.iloc[0]).drop(df.index[0]) # Quito el Index del encabezado y le pongo la fila 1
    df = df.reset_index(drop=True) # Reseteo el index porque al transponer ya no esta.
    df['VCMA-EXP'] = apca_id
    df['ALCANCE'] = alcance
    list_df_all.append(df)
  return list_df_all

# LP

## Notificación

### Excel to dataframe y saber cuantas sheet hay en el excel

In [ ]:
# Load excel
xls = pd.ExcelFile('/content/drive/MyDrive/00_Gob_Medio_Amb/20220223 Notificaciones LP_excel.xlsx')

In [ ]:
end = len(xls.sheet_names)

### Ejecutar funciones

In [ ]:
list_df_all = manipulador(xls,end, 'Notificación')

### Merge DF

In [ ]:
df_all_merge_LP_notificacion = pd.concat(list_df_all)

In [ ]:
df_all_merge_LP_notificacion.dropna(how='all', axis=1, inplace=True) #Quitar Columnas vacias

## Autorización

### Excel to dataframe y saber cuantas sheet hay en el excel

In [ ]:
# Load excel
xls = pd.ExcelFile('/content/drive/My Drive/00_Gob_Medio_Amb/20220321 Autorizaciones LP.xlsx')

In [ ]:
end = len(xls.sheet_names)

### Ejecutar funciones

In [ ]:
list_df_all = manipulador(xls, end, 'Autorización')

### Merge DF

In [ ]:
df_all_merge_LP_autorizacion = pd.concat(list_df_all)

In [ ]:
df_all_merge_LP_autorizacion.dropna(how='all', axis=1, inplace=True) #Quitar Columnas vacias

# SCT

## Notificación

### Excel to dataframe y saber cuantas sheet hay en el excel

In [ ]:
# Load excel
xls = pd.ExcelFile('/content/drive/My Drive/00_Gob_Medio_Amb/20211226 Notificaciones SCT_2.xlsx')

In [ ]:
end = len(xls.sheet_names)

### Ejecutar funciones

In [ ]:
list_df_all = manipulador(xls,end, 'Notificación')

### Merge DF

In [ ]:
df_all_merge_SCT_notificacion = pd.concat(list_df_all)

In [ ]:
df_all_merge_SCT_notificacion.dropna(how='all', axis=1, inplace=True) #Quitar Columnas vacias

## Autorización

### Excel to dataframe y saber cuantas sheet hay en el excel

In [ ]:
# Load excel
xls = pd.ExcelFile('/content/drive/My Drive/00_Gob_Medio_Amb/20220225_Autorizaciones_SCT-excel.xlsx')

In [ ]:
end = len(xls.sheet_names)

### Ejecutar funciones

In [ ]:
list_df_all = manipulador(xls,end, 'Autorización')

### Merge DF

In [ ]:
df_all_merge_SCT_autorizacion = pd.concat(list_df_all)

In [ ]:
df_all_merge_SCT_autorizacion.dropna(how='all', axis=1, inplace=True) #Quitar Columnas vacias

# Unir los data frames de LP & SCT

In [ ]:
merge = [df_all_merge_LP_notificacion, df_all_merge_LP_autorizacion, df_all_merge_SCT_notificacion, df_all_merge_SCT_autorizacion]

In [ ]:
df_all_merge = pd.concat(merge)

## Mover columnas y quitar columnas con datos innecesarios.

In [ ]:
first_column = df_all_merge.pop('VCMA-EXP')  
df_all_merge.insert(0, 'VCMA-EXP', first_column) 

# Guardar el archivo final

In [ ]:
path = '/content/drive/My Drive/00_Gob_Medio_Amb/APCA-General-ALL/All_APCA-GENERAL-lp-SCT-Merge-DF.xlsx'
with pd.ExcelWriter(path) as writer:
    # use to_excel function and specify the sheet_name and index
    # to store the dataframe in specified sheet
    df_all_merge.to_excel(writer, sheet_name="Focos_all", index=False)